In [1]:
from pathlib import Path

import cv2
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import torch.utils.data as data
import os

from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt

In [45]:
df.columns

Index(['rating', 'playback_used', 'ebird_code', 'channels', 'date', 'pitch',
       'duration', 'filename', 'speed', 'species', 'number_of_notes', 'title',
       'secondary_labels', 'bird_seen', 'sci_name', 'location', 'latitude',
       'sampling_rate', 'type', 'elevation', 'description', 'bitrate_of_mp3',
       'file_type', 'volume', 'background', 'xc_id', 'url', 'country',
       'author', 'primary_label', 'longitude', 'length', 'time', 'recordist',
       'license', 'resampled_sampling_rate', 'resampled_filename',
       'resampled_channels'],
      dtype='object')

In [2]:
df = pd.read_csv("../input/birdsong-recognition/train_32000.csv")

In [27]:
p2c = {p:c for c ,p in zip(df.ebird_code.values, df.primary_label.values)}

In [73]:
p2c

{'Acanthis flammea_Common Redpoll': 'comred',
 "Accipiter cooperii_Cooper's Hawk": 'coohaw',
 'Accipiter striatus_Sharp-shinned Hawk': 'shshaw',
 'Actitis macularius_Spotted Sandpiper': 'sposan',
 'Aechmophorus occidentalis_Western Grebe': 'wesgre',
 'Aeronautes saxatalis_White-throated Swift': 'whtswi',
 'Agelaius phoeniceus_Red-winged Blackbird': 'rewbla',
 'Aix sponsa_Wood Duck': 'wooduc',
 'Alectoris chukar_Chukar': 'chukar',
 'Amphispiza bilineata_Black-throated Sparrow': 'bktspa',
 'Anas acuta_Northern Pintail': 'norpin',
 'Anas crecca_Green-winged Teal': 'gnwtea',
 'Anas platyrhynchos_Mallard': 'mallar3',
 'Anser caerulescens_Snow Goose': 'snogoo',
 'Anthus rubescens_American Pipit': 'amepip',
 'Aphelocoma californica_California Scrub-Jay': 'cowscj1',
 "Aphelocoma woodhouseii_Woodhouse's Scrub-Jay": 'wooscj2',
 'Aquila chrysaetos_Golden Eagle': 'goleag',
 'Archilochus alexandri_Black-chinned Hummingbird': 'bkchum',
 'Archilochus colubris_Ruby-throated Hummingbird': 'rthhum',
 'A

In [30]:
len(p2c.keys())

264

In [33]:
len(list(set(p2c.values())))

264

In [28]:
p2c

{'Acanthis flammea_Common Redpoll': 'comred',
 "Accipiter cooperii_Cooper's Hawk": 'coohaw',
 'Accipiter striatus_Sharp-shinned Hawk': 'shshaw',
 'Actitis macularius_Spotted Sandpiper': 'sposan',
 'Aechmophorus occidentalis_Western Grebe': 'wesgre',
 'Aeronautes saxatalis_White-throated Swift': 'whtswi',
 'Agelaius phoeniceus_Red-winged Blackbird': 'rewbla',
 'Aix sponsa_Wood Duck': 'wooduc',
 'Alectoris chukar_Chukar': 'chukar',
 'Amphispiza bilineata_Black-throated Sparrow': 'bktspa',
 'Anas acuta_Northern Pintail': 'norpin',
 'Anas crecca_Green-winged Teal': 'gnwtea',
 'Anas platyrhynchos_Mallard': 'mallar3',
 'Anser caerulescens_Snow Goose': 'snogoo',
 'Anthus rubescens_American Pipit': 'amepip',
 'Aphelocoma californica_California Scrub-Jay': 'cowscj1',
 "Aphelocoma woodhouseii_Woodhouse's Scrub-Jay": 'wooscj2',
 'Aquila chrysaetos_Golden Eagle': 'goleag',
 'Archilochus alexandri_Black-chinned Hummingbird': 'bkchum',
 'Archilochus colubris_Ruby-throated Hummingbird': 'rthhum',
 'A

In [34]:
df.secondary_labels[0].replace('[', '').replace(']', '').replace("\'", '').split(",")

['Empidonax minimus_Least Flycatcher',
 ' Leiothlypis peregrina_Tennessee Warbler',
 ' Dumetella carolinensis_Gray Catbird',
 ' Spizella pusilla_Field Sparrow']

In [61]:
non_labels = []
sls_log = []
for sls in df.secondary_labels:
    sls_2 = []
    for sl in sls.replace('[', '').replace(']', '').replace("\'", '').replace("\"", '').split(","):
        if sl == "":
            continue
        if sl[0] == " ":
            sl = sl[1:]
        if sl[-1] == " ":
            sl = sl[:-1]
        try:
#             print(p2c[sl])
#             sls_log.append(p2c[sl])
            sls_2.append(p2c[sl])
        except:
            non_labels.append(sl)
    sls_log.append(sls_2)
#     print(sls_2)

In [71]:
labels_all = []
for i, label in enumerate(df.ebird_code.values):
    labels = [label]
    labels.extend(sls_log[i])
    labels_all.append(labels)

In [72]:
labels_all

[['aldfly', 'leafly', 'grycat', 'fiespa'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly', 'grycat', 'cedwax'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly', 'comyel', 'grycat', 'reevir1', 'houwre'],
 ['aldfly'],
 ['aldfly', 'btnwar', 'amerob', 'amered'],
 ['aldfly', 'comyel', 'amecro', 'sonspa', 'comloo'],
 ['aldfly', 'norwat', 'rewbla', 'sonspa', 'amered'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly', 'barswa', 'rewbla', 'vesspa', 'leafly', 'easkin'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly', 'leafly', 'amered', 'canwar'],
 ['aldfly'],
 ['aldfly', 'cangoo'],
 ['aldfly'],
 ['aldfly'],
 ['aldfly', 'gryfly', 'comgra', 'houfin'],
 ['aldfly', 'comyel', 'rewbla'],
 ['aldfly', 'amegfi', 'comyel', 'fiespa', 'amecro'],
 ['aldfly',
  'wilfly',
  'amerob',
  'sonspa',
  'comyel',
  'blujay',
  'amegfi',
  'balori'],
 ['aldfly'],
 ['aldfly', 'comyel', 'balori'],
 ['aldfly', 'rewbla', 'comyel', 'bkbmag1', 'amecro', 'norwat', 'blujay'],

In [52]:
len(set(non_labels))

719

In [ ]:
idx = ys.argsort()[2]
PERIOD = 6
sample = df.loc[idx, :]
wav_name = sample["resampled_filename"]
ebird_code = sample["ebird_code"]
y, sr = sf.read(os.path.join("../input/train_audio_resampled_32000/",  ebird_code ,  wav_name))

In [ ]:
len_y = len(y)
print(f"len_y {len_y}")
effective_length = sr * PERIOD
if len_y < effective_length:
    new_y = np.zeros(effective_length, dtype=y.dtype)
    start = np.random.randint(effective_length - len_y)
    new_y[start:start + len_y] = y
    y_crop = new_y.astype(np.float32)
elif len_y > effective_length:
    start = np.random.randint(len_y - effective_length)
    y_crop = y[start:start + effective_length].astype(np.float32)
else:
    y_crop = y.astype(np.float32)
print(f"start position {start}, len {len(y_crop)}")
melspec = librosa.feature.melspectrogram(y_crop, sr=sr, **{})
melspec = librosa.power_to_db(melspec).astype(np.float32)
print(f"melspec size {melspec.shape}")
plt.axis('off')
plt.imshow(melspec)

In [ ]:
len_y = len(y)
print(f"len_y {len_y}")
pad = 1000
effective_length = sr * PERIOD
y_crop = np.tile(y[pad :-pad ], (effective_length // (len_y-pad *2) + 1,))
start = np.random.randint(len(y_crop) - effective_length)
y_crop = y_crop[start:start + effective_length].astype(np.float32)
print(f"start position {start}, len {len(y_crop)}")
melspec = librosa.feature.melspectrogram(y_crop, sr=sr, **{})
melspec = librosa.power_to_db(melspec).astype(np.float32)
print(f"melspec size {melspec.shape}")
plt.axis('off')
plt.imshow(melspec)

In [ ]:
print(f"min length {min(ys)} max length {max(ys)} mean length {sum(ys) / len(ys)}")